# 01.1 Data Cleaning with Polars

This notebook demonstrates how to load, clean, and save transaction data using Polars.

In [ ]:
import os
import polars as pl

datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
tx = pl.read_csv(f"{datasets_dir}/transactions.csv")
print(f'Loaded records: {tx.shape[0]}')
tx_clean = tx.drop_nulls().filter(pl.col('amount') > 0)
print(f'After filtering negatives: {tx_clean.shape[0]} rows')
if tx_clean['date'].dtype == pl.String:
    tx_clean = tx_clean.with_columns([
        pl.col('amount').round(2).alias('amount'),
        pl.col('date').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%.f').alias('date')
    ])
else:
    tx_clean = tx_clean.with_columns([pl.col('amount').round(2).alias('amount')])
print(tx_clean.head())
tx_clean.write_csv(f'{datasets_dir}/cleaned_transactions.csv')
print(f'Saved cleaned data with {tx_clean.shape[0]} rows to {datasets_dir}/cleaned_transactions.csv')